# Cow vs Horse Classifier 

## Import Libs

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
import glob
import os

In [2]:
tf.__version__

'2.2.0'

## Preprocessing Data

### Definitions

In [3]:
rescale_default = 1./255
shear_range_default=0.2
zoom_range_default=0.2
target_size_default=(64, 64)
batch_size_default=50
class_mode_default='binary'
input_shape_default = [target_size_default[0], target_size_default[1], 3]
main_path = r'D:\Github\julio-repos\datascience\datasets\cows-and-horses'
train_folder = main_path + '\\train'
test_folder = main_path + '\\test'
validaton_folder = main_path + '\\validation'
epochs_default = 30

In [4]:
def preprocess():
    train_datagen = image.ImageDataGenerator(
          rescale=rescale_default,
          shear_range= shear_range_default,
          zoom_range=zoom_range_default,
          horizontal_flip=True)
    train_set = train_datagen.flow_from_directory(
          train_folder,
          target_size=target_size_default,
          batch_size=batch_size_default,
          class_mode=class_mode_default)
    test_datagen = image.ImageDataGenerator(rescale=rescale_default)
    test_set = test_datagen.flow_from_directory(
        test_folder,
        target_size=target_size_default,
        batch_size=batch_size_default,
        class_mode=class_mode_default)
    return train_set, test_set

### Get Test And Train Sets

In [5]:
test_set, train_set = preprocess()

Found 3030 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


## Init Model and Layers

### Methods

In [6]:
def get_cnn():
    cnn = tf.keras.models.Sequential()
    return cnn

def get_conv_layer(first=False):
    if (first):
        conv_layer = tf.keras.layers.Conv2D(
          filters=64,
          kernel_size = 3,
          activation = 'relu',
          padding='same'
          )
        return conv_layer
    conv_layer = tf.keras.layers.Conv2D(
        filters=128,
        kernel_size = 3,
        activation = 'relu',
        input_shape = input_shape_default,
        padding='same'
        )
    return conv_layer

def get_pooling_layer():
    max_pooling_layer = tf.keras.layers.MaxPool2D(pool_size=(4,4), strides=4, padding='valid')
    return max_pooling_layer

def add_flatten_layer(cnn):
    cnn.add(tf.keras.layers.Flatten())
    return cnn

def get_feature_learning_layers(cnn, num_layers=2):
    for i in range(num_layers):
        first = i == 0
        layer = get_conv_layer(first)
        pooling = get_pooling_layer()
        cnn.add(layer)
        cnn.add(pooling)
    if num_layers > 2 and i > 0:
        cnn.add(tf.keras.layers.Dropout((i * 0.1) + 0.2))
    cnn = add_flatten_layer(cnn)
    return cnn
    
def add_full_connection(cnn, num_layers=1):
    for i in range(num_layers):
        dnn_layer = tf.keras.layers.Dense(units=128, activation='relu')
        cnn.add(dnn_layer)
    if num_layers > 1:
        cnn.add(tf.keras.layers.Dropout(0.45))
    return cnn

def add_output_layer(cnn):
    outputLayer = tf.keras.layers.Dense(units=1, activation='sigmoid')
    cnn.add(outputLayer)
    return cnn

def build_full_model(num_feature_learning_layers=2, num_dnn_layers=1, shoudl_compile=True):
    cnn = get_cnn()
    cnn = get_feature_learning_layers(cnn,num_feature_learning_layers)
    cnn = add_full_connection(cnn,num_dnn_layers)
    cnn = add_output_layer(cnn)
    if shoudl_compile:
        cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return cnn

def train(cnn, train_set,test_set, epochs):
    cnn.fit(x= train_set, validation_data= test_set, epochs=epochs)

## Build, Training and Evaluating

In [7]:
cnn = build_full_model(num_feature_learning_layers=3, num_dnn_layers=1)
train(cnn, train_set,test_set, epochs_default)

Epoch 1/30
8/8 [==============================] - 8s 977ms/step - loss: 0.7018 - accuracy: 0.5050 - val_loss: 0.6959 - val_accuracy: 0.5000
Epoch 2/30
8/8 [==============================] - 8s 991ms/step - loss: 0.6979 - accuracy: 0.5000 - val_loss: 0.6921 - val_accuracy: 0.5023
Epoch 3/30
8/8 [==============================] - 8s 996ms/step - loss: 0.6900 - accuracy: 0.5225 - val_loss: 0.6917 - val_accuracy: 0.5109
Epoch 4/30
8/8 [==============================] - 8s 1s/step - loss: 0.6848 - accuracy: 0.5475 - val_loss: 0.6899 - val_accuracy: 0.5667
Epoch 5/30
8/8 [==============================] - 8s 993ms/step - loss: 0.6845 - accuracy: 0.5575 - val_loss: 0.6848 - val_accuracy: 0.6145
Epoch 6/30
8/8 [==============================] - 8s 1s/step - loss: 0.6775 - accuracy: 0.5900 - val_loss: 0.6764 - val_accuracy: 0.6389
Epoch 7/30
8/8 [==============================] - 8s 1s/step - loss: 0.6719 - accuracy: 0.6000 - val_loss: 0.6677 - val_accuracy: 0.6436
Epoch 8/30
8/8 [=============

## Predictions

In [8]:
def predict_item(model, image_path, target_size):
    image_predict = image.load_img(image_path, target_size= target_size)
    image_predict = image.img_to_array(image_predict)
    image_predict = np.expand_dims(image_predict,axis=0)  # Convert single image to a batch.
    predictions = model.predict(image_predict)
    return predictions

def predict_class(train_set, predictions):
    indices = train_set.class_indices
    reverse_indices = {}
    for x, y in indices.items():
        reverse_indices[str(y)] = x
    idx = int(predictions[0][0])
    return reverse_indices[str(idx)]

def predict_many(model, train_set, folder_path, target_size, expected_class):
    images_paths = glob.glob(folder_path)
    predictions_list = []
    for image_path in images_paths:
        predictions = predict_item(model, image_path, target_size)
        class_predicted = predict_class(train_set, predictions)
        prediction = {
            'expected':expected_class.lower(),
            'found': class_predicted.lower(),
            'got_it_rigth': expected_class.lower() == class_predicted.lower()
            }
        predictions_list.append(prediction)
    return predictions_list

def print_predictions(predictions_list):
    rights = filter(lambda item: item['got_it_rigth'] == True, predictions_list)
    _percent_right = len(list(rights)) / len(predictions_list)
    print("% righs {}".format(_percent_right * 100))
    for idx, item in enumerate(predictions_list):
        print("#{} Expected: {} - Found: {} - Right? {}".format(idx,item['expected'], item['found'], item['got_it_rigth']))

In [9]:
validation_cows = validaton_folder + '/cows/*'
validation_horses = validaton_folder + '/horses/*'

In [10]:
cows_result = predict_many(cnn, train_set, validation_cows, target_size_default,'cows')
horses_result = predict_many(cnn, train_set, validation_horses, target_size_default,'horses')

In [11]:
print_predictions(cows_result)

% righs 91.3907284768212
#0 Expected: cows - Found: horses - Right? False
#1 Expected: cows - Found: cows - Right? True
#2 Expected: cows - Found: cows - Right? True
#3 Expected: cows - Found: cows - Right? True
#4 Expected: cows - Found: cows - Right? True
#5 Expected: cows - Found: cows - Right? True
#6 Expected: cows - Found: cows - Right? True
#7 Expected: cows - Found: cows - Right? True
#8 Expected: cows - Found: cows - Right? True
#9 Expected: cows - Found: cows - Right? True
#10 Expected: cows - Found: cows - Right? True
#11 Expected: cows - Found: cows - Right? True
#12 Expected: cows - Found: cows - Right? True
#13 Expected: cows - Found: cows - Right? True
#14 Expected: cows - Found: cows - Right? True
#15 Expected: cows - Found: cows - Right? True
#16 Expected: cows - Found: cows - Right? True
#17 Expected: cows - Found: cows - Right? True
#18 Expected: cows - Found: cows - Right? True
#19 Expected: cows - Found: cows - Right? True
#20 Expected: cows - Found: cows - Right? 

In [12]:
print_predictions(horses_result)

% righs 55.66502463054187
#0 Expected: horses - Found: horses - Right? True
#1 Expected: horses - Found: horses - Right? True
#2 Expected: horses - Found: horses - Right? True
#3 Expected: horses - Found: horses - Right? True
#4 Expected: horses - Found: cows - Right? False
#5 Expected: horses - Found: cows - Right? False
#6 Expected: horses - Found: horses - Right? True
#7 Expected: horses - Found: cows - Right? False
#8 Expected: horses - Found: horses - Right? True
#9 Expected: horses - Found: cows - Right? False
#10 Expected: horses - Found: horses - Right? True
#11 Expected: horses - Found: horses - Right? True
#12 Expected: horses - Found: horses - Right? True
#13 Expected: horses - Found: cows - Right? False
#14 Expected: horses - Found: cows - Right? False
#15 Expected: horses - Found: cows - Right? False
#16 Expected: horses - Found: cows - Right? False
#17 Expected: horses - Found: horses - Right? True
#18 Expected: horses - Found: horses - Right? True
#19 Expected: horses - 